<font size ="5"><b>INFO 370 Assignment 8: Applied Modeling</b></font>

Name: Ellen Gorbach

<font size ="3"><b>1: Is COMPAS fair?</b></font>

In [1]:
#1.1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

compasAllData = pd.read_csv("data/compas-score-data.csv.bz2", sep="\t")
columns = len(compasAllData.columns)
print(columns)
rows = len(compasAllData)
print(rows)
numNull = compasAllData.isna().sum()
print(numNull)
compasAllData.head()

8
6172
age                0
c_charge_degree    0
race               0
age_cat            0
sex                0
priors_count       0
decile_score       0
two_year_recid     0
dtype: int64


,age,c_charge_degree,race,age_cat,sex,priors_count,decile_score,two_year_recid
0,69,F,Other,Greater than 45,Male,0,1,0
1,34,F,African-American,25 - 45,Male,0,3,1
2,24,F,African-American,Less than 25,Male,4,4,1
3,44,M,Other,25 - 45,Male,0,1,0
4,41,F,Caucasian,25 - 45,Male,14,6,1


In [2]:
#1.2 
compas = compasAllData[(compasAllData.race == "African-American") | 
           (compasAllData.race == "Caucasian")]
compas.head()

,age,c_charge_degree,race,age_cat,sex,priors_count,decile_score,two_year_recid
1,34,F,African-American,25 - 45,Male,0,3,1
2,24,F,African-American,Less than 25,Male,4,4,1
4,41,F,Caucasian,25 - 45,Male,14,6,1
6,39,M,Caucasian,25 - 45,Female,0,1,0
7,27,F,Caucasian,25 - 45,Male,0,4,0


In [3]:
#1.3
# low risk = 1-4 high risk = 5-10 based on decile_score
compas["decileRisk"] = pd.cut(compas.decile_score,
                            bins = [0, 4, 10],
                            labels = [0, 1])
# 0 is low risk and 1 is high risk of recidivism
compas.head()

<ipython-input-3-5b8e3d73f960>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compas["decileRisk"] = pd.cut(compas.decile_score,


,age,c_charge_degree,race,age_cat,sex,priors_count,decile_score,two_year_recid,decileRisk
1,34,F,African-American,25 - 45,Male,0,3,1,0
2,24,F,African-American,Less than 25,Male,4,4,1,0
4,41,F,Caucasian,25 - 45,Male,14,6,1,1
6,39,M,Caucasian,25 - 45,Female,0,1,0,0
7,27,F,Caucasian,25 - 45,Male,0,4,0,0


In [4]:
#1.4 a
riskRate = compas.groupby("decileRisk").two_year_recid.value_counts(normalize=True) * 100
print(riskRate)
# Recidivism rate for low-risk group is 32.001453% and recidivism rate for high-risk group is 63.445545%.

decileRisk  two_year_recid
0           0                 67.998547
            1                 32.001453
1           1                 63.445545
            0                 36.554455
Name: two_year_recid, dtype: float64


In [5]:
#1.4 b
raceRate = compas.groupby("race").two_year_recid.value_counts(normalize=True) * 100
print(raceRate)
# Recidivism rate for African Americans is 52.314961% and recidivism rate for Caucasians is 39.087019%.

race              two_year_recid
African-American  1                 52.314961
                  0                 47.685039
Caucasian         0                 60.912981
                  1                 39.087019
Name: two_year_recid, dtype: float64


In [6]:
#1.5
from sklearn.metrics import confusion_matrix
# recidivists = positives
cm  = confusion_matrix(compas.two_year_recid,compas.decileRisk)
cm
#         | pre low  | pre high 
# no rec  | 1872     | 923  | N = 2795
# act rec | 881      | 1602 | P = 2483
#         | 2753     | 2525 | T = 5278
# 35.48% of recidivists were falsely classified as low-risk (FNR). 
# 33.02% of non-recidivists were falsely classified as high-risk (FPR).
# COMPAS accurately classified 65.82% of individuals. 
# The percentage of people that were accurately classified as high-risk is 63.45%. 
# The percentage of recidivists accurately classified is 64.52%. 

array([[1872,  923],
       [ 881, 1602]])

In [7]:
#1.6
# I definitely would not feel comfortable having a judge use COMPAS with this high of a percentage for the various
# misclassifications/errors. I'm most concerned about the 33.02% of non-recidivists that were classified as high
# risk because that means such a high percentage of people are potentially being placed with longer sentences than
# necessary. Algorithms are frequently seen as infallible so the results from COMPAS could make a judge much more
# convinced that the sentence they are given is "correct," so COMPAS needs to be held to a higher standard than
# a judge who is human and seen as capable of making mistakes. # I truly believe that all of the errors would need
# to be below 2% in order for COMPAS to be acceptable to use.

In [8]:
#1.7 a&b&c African-American
compasAA = compas[compas.race == "African-American"]
# recidivists = positives
cmAA  = confusion_matrix(compasAA.two_year_recid,compasAA.decileRisk)
cmAA
#         | pre low  | pre high 
# no rec  | 873      | 641  | N = 1514
# act rec | 473      | 1188 | P = 1661
#         | 1346     | 1829 | T = 3175

# COMPAS accurately classified 64.91% of individuals.
# The FPR (the rate of people falsely classified as high-risk of recidivism) is 0.4234 or 42.34%.
# The FNR (the rate of people falsely classified as low-risk of recidivism) is 0.2848 or 28.48%.

array([[ 873,  641],
       [ 473, 1188]])

In [9]:
#1.7 a&b&c Caucasian

compasC = compas[compas.race == "Caucasian"]
# recidivists = positives
cmC = confusion_matrix(compasC.two_year_recid,compasC.decileRisk)
cmC
#         | pre low  | pre high 
# no rec  | 999      | 282  | N = 1281
# act rec | 408      | 414 | P = 822
#         | 1407     | 696 | T = 2103

# COMPAS accurately classified 67.19% of individuals.
# The FPR (the rate of people falsely classified as high-risk of recidivism) is 0.2201 or 22.01%.
# The FNR (the rate of people falsely classified as low-risk of recidivism) is 0.4964 or 49.64%.

array([[999, 282],
       [408, 414]])

In [10]:
#1.8
# I believe that the COMPAS algorithm is not fair. I chose to define fair being able to ensure similar FPR and
# FNR across categories, which in this case is race. As we can see, Causasians have an FPR of approximately 22%
# while African Americans have an FPR of approximately 42%. African Americans are 20% more likely to be falsely 
# predicted to recidivate, resulting in inequality in the sentences and treatmeants doled out by the courts. We 
# can also see that Caucasians have an FNR of approximately 49% while African Americans have an FNR of
# approximately 28% which means Caucasians are 21% more likely to be predicted to not recidivate, resulting in
# less harsh sentences. There's a clear divide in the treatment of African Americans and Caucasians in the 
# COMPAS algorithm that skews towards punishing African Americans, even though their classification percentages
# are fairly similar, showing that something is inherently unbalanced in the algorithm. For recidivism rates
# that are only 13% apart, the differences in FPR and FNR between the two races is drastically too large for
# this algorithm to be considered fair.

<font size ="3"><b>2: Can you beat COMPAS?</b></font>

In [11]:
#2.1

# Although all of this information is incredibly important, I believe that the false negative case (aka predicted
# recidivism but the person recidivate) is the most detrimental. This results in certain people spending more time
# in prison and especially since there is a racial skew towards African-Americans in this false negative means
# that this is the problem we should be trying to minimize. I believe that it's better to let a guilty person go
# free than to jail an innocent person so our first goal is to measure recall (since it's a good measure for
# false negatives) with a second goal of precision (since we also don't want to let guilty people walk free).
# I don't want to use F-score because I want to have recall weigh more in my conclusions than precision.

In [12]:
#2.2 
compas["priorsCat"] = pd.cut(compas.priors_count,  # creating dummy variables
                            bins = [-np.inf, 0, 10, 20, np.inf],
                            labels = ["0", "1-10", "11-20", "21+"])
priors = pd.get_dummies(compas.priorsCat, drop_first=True)
age = pd.get_dummies(compas.age_cat, drop_first=True)
degree = pd.get_dummies(compas.c_charge_degree, drop_first=True)
risk = pd.get_dummies(compas.decileRisk, drop_first=True)

from sklearn.linear_model import LogisticRegression # modeling
X = pd.concat((priors, risk, age, degree), axis = 1)
y = compas.two_year_recid

m = LogisticRegression()

from sklearn.model_selection import cross_val_score
recallCV = (cross_val_score(m,X,y,scoring="recall_weighted",cv=10)).mean()
print(recallCV)
precisionCV = (cross_val_score(m,X,y,scoring="precision_weighted",cv=10)).mean()
print(precisionCV)

# Used weighted for precision and recall because it gets the precision and recall for each fold. Source below.
# https://www.researchgate.net/post/How_to_find_the_metrics_for_cross_folds_validation_process

<ipython-input-12-61f972605a0d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compas["priorsCat"] = pd.cut(compas.priors_count,  # creating dummy variables


0.6742988471048244
0.67389767234811


In [13]:
#2.3
compas["ageTest"] = pd.cut(compas.age,  # creating dummy variables
                            bins = [-np.inf, 19, 21, 28, 40, np.inf],
                            labels = ["<=19", "20-21", "22-28", "29-40", "41+"])
age2 = pd.get_dummies(compas.ageTest, drop_first=True)

compas["priorTest"] = pd.cut(compas.priors_count,  # creating dummy variables
                            bins = [-np.inf, 0, 5, 9, 11, 20, 30, np.inf],
                            labels = ["0", "1-5","6-9", "10-11", "12-20", "21-30", "31+"])
priors2 = pd.get_dummies(compas.priorTest, drop_first=True)


X = pd.concat((priors2, risk, age2, degree), axis = 1)
y = compas.two_year_recid

m = LogisticRegression()


recallCV = (cross_val_score(m,X,y,scoring="recall_weighted",cv=10)).mean()
print(recallCV)
precisionCV = (cross_val_score(m,X,y,scoring="precision_weighted",cv=10)).mean()
print(precisionCV)

# I attempted to remove different features.
# I attempted various groupings for categories for ages and priors. 
# The ones above are the best picks that I could find.
# COMPAS had the recall as .6452 and I have .6821 so my model is better for recall.
# COMPAS had the precision as 0.6345 and I have .6821 so my model is also better for precison.
# Considering that both performance measurements I used are better, I can say my model is better than COMPAS.

<ipython-input-13-d001c87838b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compas["ageTest"] = pd.cut(compas.age,  # creating dummy variables
<ipython-input-13-d001c87838b4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compas["priorTest"] = pd.cut(compas.priors_count,  # creating dummy variables


0.6820704674831809
0.6820718193057329


In [14]:
#2.4 adding sex

sex = pd.get_dummies(compas.sex, drop_first=True)
X = pd.concat((priors2, risk, age2, degree, sex), axis = 1)
y = compas.two_year_recid

m = LogisticRegression()

recallCV = (cross_val_score(m,X,y,scoring="recall_weighted",cv=10)).mean()
print(recallCV)
precisionCV = (cross_val_score(m,X,y,scoring="precision_weighted",cv=10)).mean()
print(precisionCV)

# Adding sex as a feature made the model worse in terms of recall and precision as compared to my model.

0.6773327439480192
0.6773474240356008


In [15]:
#2.5 a

race = pd.get_dummies(compas.race, drop_first=True)
X = pd.concat((priors2, risk, age2, degree, sex, race), axis = 1)
y = compas.two_year_recid

m = LogisticRegression()

recallCV = (cross_val_score(m,X,y,scoring="recall_weighted",cv=10)).mean()
print(recallCV)
precisionCV = (cross_val_score(m,X,y,scoring="precision_weighted",cv=10)).mean()
print(precisionCV)

# Adding race as a feature made the model worse than just having sex and the model did not reach
# the level at which I had it without sex and race (in terms of recall and precision).

0.6769543154505204
0.6769612293920081


In [16]:
# 2.5 b
# Out of curiosity, I decided to try adding race to the model but not sex.

X = pd.concat((priors2, risk, age2, degree, race), axis = 1)
y = compas.two_year_recid

m = LogisticRegression()

recallCV = (cross_val_score(m,X,y,scoring="recall_weighted",cv=10)).mean()
print(recallCV)
precisionCV = (cross_val_score(m,X,y,scoring="precision_weighted",cv=10)).mean()
print(precisionCV)

# Adding race (but not sex) as a feature made the model better than my best attempt in terms of precision 
# and recall.

0.6824492553619688
0.6825481173993606


In [17]:
#2.6 

# I managed to surpass COMPAS in its predictive capabilities when looking at recall and precision as performance
# measures. Race on its own helped improve my predictions but sex on its own and race and sex combined made it 
# worse. I don't think either my model or COMPAS are accurate enough in their predictions to be useful tools for 
# predicting recidivism and that judges, if they do decide to use these models, should take the results from them 
# with a high grain of salt. Especially if the features that went into the model are not public information, its
# almost impossible to tell what sort of biases are present in it and will be perpetuated if the model is used
# for substantial decisions like sentence lengths for criminals.

I spent around 6.5 hours on this assignment